In [154]:
## variaveis iniciais e de configuração
## win_resul armazena as posições de todas as vitorias
## last_move indica em qual quadrado deverá ser a proxima jogada(-1 significa qualquer lugar)
win_resul = [(0,4,8),(2,4,6)]
win_resul += [(i, i+3, i+6) for i in range(3)]
win_resul += [(i*3, i*3+1, i*3+2) for i in range(3)]
last_move = -1

## cria e preenche o tabuleiro
def create_board():
    board = []
    square = ['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']
    for i in range(9):
        board.append(square.copy())
    return board

## imprime o tabuleiro
def print_board(board):
    for h in range(3):
        print("-"*25)
        for i in range(3):
            line = "| "
            for j in range(3):
                line += board[j+h*3][i*3] + " " + board[j+h*3][i*3+1] + " " + board[j+h*3][i*3+2] + " | "
            print(line)
    print("-"*25)

## verifica se há vencedor num dado quadrado
def small_win(square):
    for i in win_resul:
        (x, y, z) = i
        square_x = square[x]
        if(square_x == square[y] == square[z]) and (square[x] != 'L'):
            return square_x
    return 'L'

## verifica se há vencedor no tabuleiro
def big_win(board):
    for i in win_resul:
        (x, y, z) = i
        small_x = small_win(board[x])
        if(small_x == small_win(board[y]) == small_win(board[z])) and (small_win(board[x]) != 'L'):
            return small_x
    return 'L'

## recebe o tabuleiro, as posições da jogada e o jogador, e retorna se a jogada é válida
## x é o numero do quadrado a ser jogado e y a posição nele
def player_move(board, x, y, player):
    if(board[x][y] == 'L'):
        return False, board
    else:
        last_move = y ## atualiza o ultimo movimento pra sabermos qual será o próximo quadrado a ser jogado
        board[x][y] = player
        return True, board

## verifica se o quadrado está todo preenchido e retorna um boolean correspondente
def full_board(board):
    for i in board[last_move]:
        if(i=='L'):
            return False
    return True

## verifica se no quadrado indicado pela ultima jogada há vitoria ou empate, caso haja last_move passa a ser -1
def next_move(board):
    target_square = board[last_move]
    if(small_win(target_square) or full_board(target_square)):
        last_move = -1

In [155]:
## testes

"""
print_board(big_ttt)
big_ttt[1][0] = 'X'
big_ttt[8][0] = 'X'
big_ttt[8][8] = 'X'
print_board(big_ttt)
"""

-------------------------
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
-------------------------
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
-------------------------
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
-------------------------
True
-------------------------
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
-------------------------
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
-------------------------
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
| F F F | F F F | F F X | 
-------------------------


"\nprint_board(big_ttt)\nbig_ttt[1][0] = 'X'\nbig_ttt[8][0] = 'X'\nbig_ttt[8][8] = 'X'\nprint_board(big_ttt)\n"